In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift, DBSCAN, estimate_bandwidth

In [2]:
np.random.seed(42)

In [3]:
df=pd.read_csv('df_PCA.csv',delimiter=',')
df.head()

,customer_id,PC0,PC1,PC2,PC3,PC4,PC5,customer_region_2440,customer_region_2490,customer_region_4140,customer_region_4660,customer_region_8370,customer_region_8550,customer_region_8670,customer_region_Unknown,last_promo_DISCOUNT,last_promo_FREEBIE,last_promo_no_promotion_used,payment_method_CASH,payment_method_DIGI
0,1b8f824d5e,0.136546,0.000247,0.588527,0.788868,0.054088,0.421970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,5d272b9dcb,0.136468,0.000000,0.591727,0.801303,0.036836,0.204790,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,f6d1b2ba63,0.214522,0.021476,0.295363,0.871453,0.354529,0.215721,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,180c632ed8,0.141770,0.006605,0.594165,0.797071,0.174897,0.281806,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4eb37a6705,0.142061,0.006840,0.594022,0.787458,0.085230,0.445947,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [4]:
df.set_index('customer_id',inplace=True)

In [5]:
metric_features=df.columns

In [27]:
bandwidth = estimate_bandwidth(df[metric_features], quantile=0.087, random_state=42, n_jobs=-1)
bandwidth

1.0095703696820082

In [33]:
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, n_jobs=4)
ms_labels = ms.fit_predict(df[metric_features])

ms_n_clusters = len(np.unique(ms_labels))
print("Number of estimated clusters : %d" % ms_n_clusters)

Number of estimated clusters : 9


In [35]:
ms_labels

array([4, 3, 2, ..., 4, 3, 5])

In [40]:
df_clean=pd.read_csv('df_Clean.csv',delimiter=',')

In [42]:
len(df_clean)

30554

In [43]:
df_clean.set_index('customer_id',inplace=True)

In [44]:
df_clean_numeric = df_clean.select_dtypes(include=['number'])
df_concat = pd.concat((df_clean_numeric, pd.Series(ms_labels, name='labels', index=df_clean.index)), axis=1)
df_concat.groupby('labels').mean()

,customer_age,vendor_count,product_count,is_chain,first_order,last_order,CUI_American,CUI_Asian,CUI_Beverages,CUI_Cafe,...,HR_22,HR_23,Total_Spent,order_count,Diversity_Cuisine,costumer_time,Intensity_of_Activity,Customer_loyalty,product_intensity,Day_Week_Spread
labels,,,,,,,,,,,,,,,,,,,,,
0,27.492334,3.543407,6.848601,3.247125,24.040437,68.268877,6.084936,11.728074,2.548389,1.066183,...,0.047144,0.035837,46.946123,5.254983,2.080258,44.228440,10.266201,0.789038,1.302173,0.893952
1,27.357002,2.865902,4.848101,2.502176,30.815071,61.162579,3.555103,12.390771,3.656226,0.081357,...,0.035799,0.040546,35.916509,3.666930,1.639052,30.347508,7.829438,0.888494,1.347203,0.595926
2,27.490099,2.334866,4.428925,1.884017,34.206506,61.203324,6.014222,4.528964,0.000000,1.848320,...,0.021570,0.011669,32.595163,3.556223,1.708623,26.996818,6.368551,0.826385,1.245514,0.591837
3,27.466995,2.849918,4.607553,2.669622,30.456486,59.766502,3.199438,7.662644,2.401635,0.111199,...,0.050246,0.033826,27.818499,3.616092,1.567448,29.310016,7.604337,0.891519,1.279739,0.588474
4,27.540993,2.982346,5.318671,2.791442,29.470377,62.785757,3.900491,8.281149,1.981403,0.673555,...,0.040694,0.035308,33.730575,4.111909,1.720478,33.315380,8.148070,0.855196,1.313228,0.669958
5,27.530379,2.539855,4.332776,2.294314,33.225195,58.027035,3.522366,7.441586,2.000499,0.599234,...,0.039298,0.028707,29.009819,3.373467,1.561971,24.801839,6.163921,0.885699,1.281430,0.557188
6,27.296263,2.718323,5.390155,2.240656,29.631723,65.216044,7.221668,5.260264,0.013327,2.403437,...,0.019143,0.011851,39.057393,4.292616,1.918737,35.584321,8.562473,0.794663,1.252112,0.760733
7,27.645938,3.818455,6.602808,4.285858,23.320963,64.676028,1.923551,2.396991,2.137151,0.307954,...,0.093280,0.065196,23.074293,5.164493,1.687931,41.355065,8.976968,0.847521,1.265579,0.875675
8,27.139394,1.842424,3.775758,0.842424,37.169697,62.200000,4.424727,1.741576,0.000000,0.497273,...,0.060606,0.024242,27.595818,2.981818,1.853333,25.030303,7.384740,0.775889,1.294566,0.485137


In [45]:
cluster_means = df_concat.groupby('labels').mean().T
cluster_means.style.format(precision=2).background_gradient(axis=1)

labels,0,1,2,3,4,5,6,7,8
customer_age,27.49,27.36,27.49,27.47,27.54,27.53,27.30,27.65,27.14
vendor_count,3.54,2.87,2.33,2.85,2.98,2.54,2.72,3.82,1.84
product_count,6.85,4.85,4.43,4.61,5.32,4.33,5.39,6.60,3.78
is_chain,3.25,2.50,1.88,2.67,2.79,2.29,2.24,4.29,0.84
first_order,24.04,30.82,34.21,30.46,29.47,33.23,29.63,23.32,37.17
last_order,68.27,61.16,61.20,59.77,62.79,58.03,65.22,64.68,62.20
CUI_American,6.08,3.56,6.01,3.20,3.90,3.52,7.22,1.92,4.42
CUI_Asian,11.73,12.39,4.53,7.66,8.28,7.44,5.26,2.40,1.74
CUI_Beverages,2.55,3.66,0.00,2.40,1.98,2.00,0.01,2.14,0.00
CUI_Cafe,1.07,0.08,1.85,0.11,0.67,0.60,2.40,0.31,0.50
